### Load test data:

In [19]:
# %load create_test_data.py
import numpy as np


" grid"
size = 101
x = np.linspace(0, 100, size)
y = np.linspace(0, 100, size)
X, Y = np.meshgrid(x, y)

" backround"
z = np.random.rand(size,size)
data= z.copy()


" create lines "

" fat line 1"
i_index = np.linspace(10, 20, 41)
j_index = np.linspace(10, 30, 41)
i_index2 = np.linspace(10, 20, 41)
j_index2 = np.linspace(11, 31, 41)

" line 2"
h_index = np.linspace(60, 80, 41)
k_index = np.linspace(60, 80, 41)

" put lines in spectrogram"
for i in range(np.shape(i_index)[0]):
    data[int(i_index[i]), int(j_index[i])] = 1.3
    data[int(i_index2[i]), int(j_index2[i])] = 1.1
    data[int(h_index[i]), int(k_index[i])] = 1.3
    
#np.save("spectrogram_data/test_data", data)

### load spectrogram plot funtion

In [34]:
# %load plot_spectrogram.py
"""
Created on Thu Jan 28 11:28:35 2016

@author: claessen
"""

import matplotlib.pyplot as plt

def plot_spectrogram(X, Y, Z, figno, clabel):
    
    plt.figure(figno)
    plt.contourf(X, Y, Z, 20)
    cbar = plt.colorbar()
    cbar.ax.set_ylabel(clabel)

    plt.title("spectrogam")
    plt.xlabel("time /s")
    plt.ylabel("frequency /Hz")
    plt.show()
    
    return

plot test data

In [35]:
plot_spectrogram(X, Y, data, 1, "Power")

filter by substracting some backround

In [36]:
rate = data.copy()/np.average(z)
plot_spectrogram(X, Y, rate, 2, "rate")

give score to datapoints

In [37]:
aplus = 10.0
aminus = 0.3
bplus = 1.5
bminus = 1.0
t = 1.5

s = np.zeros(np.shape(z))
s[np.where(rate>=t)] = aplus*(rate[np.where(rate>=t)]-t)**bplus
s[np.where(rate<t)] = aminus*(t-rate[np.where(rate<t)])**bminus

plot_spectrogram(X, Y, s, 3, "score")

## Furses algorithm

line object

In [38]:
class line:
    def __init__(self, startpos, startfreq, score):
        self.scorelist = []
        self.scorelist.append(score)
        
        self.startpos = startpos

        self.startfreq = startfreq
        self.curpos = self.startpos
        self.curfreq = self.startfreq
        
        self.slope = 0
        self.duration = 0

        
    def investigate_point(self, pos, freqin, curslice):
        
        success = 0
        
            
        if np.abs(pos-self.curpos)<freq_delta and np.abs(freqin -self.curfreq) < freq_delta:
    
            freqin, score = converge_function(freqin, curslice)
                
            self.curpos = pos
            self.curfreq = freqin
            self.duration = self.curpos - self.startpos
            self.slope = np.abs(self.curfreq-self.startfreq)/(self.curpos-self.startpos)
            self.scorelist.append(score)
            success = 1
            
        return success, freqin 

function to inspect nearby datapoint and recalculated weighed frequency

In [39]:
def converge_function(f, s1):
    delta = freq_delta + 1
    loop_counter = 0
    while np.abs(delta) > freq_delta:
        loop_counter += 1
        if loop_counter > 5:
            print "loop error"
            break
        
        else:

            weighing = s1[round(f)-freq_delta:round(f)+freq_delta+1] / np.sum(s1[round(f)-freq_delta:round(f)+freq_delta+1])
            score = np.sum(s[f-freq_delta:f+freq_delta+1])
            new_freq = (np.sum(weighing*np.linspace(f-freq_delta,f+freq_delta, 2*freq_delta+1)))
            delta = np.abs(new_freq-f)
            f = new_freq
                    
    f1 = f
    global slice1
    slice1[round(f)-freq_delta:round(f)+freq_delta+1] = -sigma
    
    return f1, score

find lines in instreaming data slices

In [40]:
tresh = np.max(s)*0.3
freq_delta = 3
sigma = -5.0
lan = 10.0

candidates = {}

counter = 0

for ii in range(size):
    freq = 0
    slice1 = s[:, ii]
    freq_list = np.where(slice1>=tresh)
    
    if len(slice1[freq_list]) > 0:
                    
        for ll in range(len(slice1[freq_list])):    
            old_freq = freq
            freq = freq_list[0][ll]
            
                        
            if np.abs(old_freq-freq) > freq_delta and freq < size-freq_delta and freq > freq_delta:
                success = 0 
                
                for jj in range(len(candidates)):
                    success, freq1 = candidates[str(jj)].investigate_point(ii, freq, slice1) 

                    if success == 1:
                        freq = freq1
                        break
                    
                if success == 0:
                    freq, score = converge_function(freq, slice1)
                    candidates[str(counter)] = line(ii,round(freq), score)
                    
                    counter += 1
    
                    
                
                               
candidates = {k:v for (k,v) in candidates.iteritems() if (v.slope!=0 and v.duration>=lan)}
          

for mm in candidates.keys():

    print candidates[mm].startpos, candidates[mm].startfreq, candidates[mm].slope, candidates[mm].curpos

10 10.0 0.470967574503 31
60 60.0 1.00727508886 80
